# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision.transforms import v2 as transformsV2
from torch.utils.data import DataLoader
from Dataset import MyDataset
from tqdm import tqdm
from Network import MyNetwork
import os
%load_ext autoreload
%autoreload 2

os.environ['KMP_DUPLICATE_LIB_OK']='True' # To prevent the kernel from dying

# DEFINITIONS

In [2]:
MEAN = np.array([0.5750, 0.6065, 0.6459])
STD = np.array([0.1854, 0.1748, 0.1794])
ROOT_DIRTECTORY = "c:\\Users\\jacop\\Desktop\\BSc\\Code\\WindTurbineImagesCategorization\\Data\\Dataset"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device in use: {DEVICE}")

Device in use: cuda:0


# HELPER FUNCTIONS

In [3]:
def create_tqdm_bar(iterable, desc):
    return tqdm(enumerate(iterable), total=len(iterable), ncols=150, desc=desc)

# TRANSFORMS

In [4]:
transform = transformsV2.Compose([
    transformsV2.Resize((224, 224)),
    transformsV2.RandomHorizontalFlip(p=0.5),
    transformsV2.RandomVerticalFlip(p=0.5),
    transformsV2.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
    transformsV2.RandomAutocontrast(p=0.5),
    transformsV2.RandomRotation(degrees=[0, 90]),
    transformsV2.ColorJitter(brightness=0.25, saturation=0.20),
    # Replace deprecated ToTensor()
    transformsV2.ToImage(),
    transformsV2.ToDtype(torch.float32, scale=True),
    transformsV2.Normalize(mean=MEAN.tolist(), std=STD.tolist()),
])

# HYPER PARAMETERS

In [10]:
hyper_parameters = {
    "input channels": 3,
    "number of classes": 5,
    "split": {"train": 0.6, "val": 0.2, "test": 0.2},
    "batch size": 16,
    "number of workers": 0,
    "learning rate": 0.001,
    "epochs": 10,
    "beta1": 0.9,
    "beta2": 0.999,
    "epsilon": 1e-08,
    "weight decay": 1e-08,
    "step size": 30,
    "gamma": 0.8,
}

# DATASETS AND DATALOADERS

In [6]:
# Create Datasets and Dataloaders
dataset_train = MyDataset(root_directory=ROOT_DIRTECTORY, mode="train",
                          transform=transform, split=hyper_parameters["split"])
print(f"Created a new Dataset for training of length: {len(dataset_train)}")
dataset_validation = MyDataset(root_directory=ROOT_DIRTECTORY,
                               mode="val", transform=None, split=hyper_parameters["split"])
print(f"Created a new Dataset for validation of length: {
      len(dataset_validation)}")

dataloader_train = DataLoader(
    dataset_train, batch_size=hyper_parameters["batch size"], shuffle=True, num_workers=hyper_parameters["number of workers"], drop_last=False)
print(f"Created a new Dataloader for training with batch size: {
      hyper_parameters["batch size"]}")
# dataloader_validation = DataLoader(dataset_validation, batch_size=hyper_parameters["batch size"], shuffle=True, num_workers=hyper_parameters["number of workers"], drop_last=False)
dataloader_validation = DataLoader(dataset_validation, batch_size=1, shuffle=False,
                                   num_workers=hyper_parameters["number of workers"], drop_last=False)
print(f"Created a new Dataloader for validation with batch size: {
      hyper_parameters["batch size"]}")

Created a new Dataset for training of length: 1320
Created a new Dataset for validation of length: 440
Created a new Dataloader for training with batch size: 16
Created a new Dataloader for validation with batch size: 16


# MODEL CREATION

In [7]:
# Create Model, initialize its weights, create optimizer and loss function
model = MyNetwork(hyper_parameters)
model.weight_initialization()
model.to(DEVICE)
print(f"Model created, move to {DEVICE} and weights initialized")

optimizer = optim.Adam(model.parameters(),
                       lr=hyper_parameters["learning rate"],
                       betas=(hyper_parameters["beta1"],
                              hyper_parameters["beta2"]),
                       weight_decay=hyper_parameters["weight decay"],
                       eps=hyper_parameters["epsilon"])
loss_function = nn.CrossEntropyLoss()

# Empty memory before start
if torch.cuda.is_available():
    torch.cuda.empty_cache()

Model created, move to cuda:0 and weights initialized


# TRAIN

In [8]:
# def train_classifier(classifier, train_loader, val_loader, loss_func, tb_logger, epochs=10, name="default"):
def train_net(model, loss_function, device, dataloader_train, dataloader_validation, optimizer, hyper_parameters, name="default"):
    epochs = hyper_parameters["epochs"]

    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size=hyper_parameters["step size"], gamma=hyper_parameters["gamma"])
    validation_loss = 0

    for epoch in range(epochs):  # loop over the dataset multiple times

        """    Train step for one batch of data    """
        training_loop = create_tqdm_bar(
            dataloader_train, desc=f'Training Epoch [{epoch+1}/{epochs}]')
        training_loss = 0

        for train_iteration, batch in training_loop:
            model.train()  # Set the model to training mode
            optimizer.zero_grad()  # Reset the parameter gradients for the current minibatch iteration

            images, labels = batch
            labels -= 1  # Change the labels to start from 0
            labels = labels.type(torch.LongTensor)

            labels = labels.to(device)
            images = images.to(device)

            # Forward pass, backward pass and optimizer step
            predicted_labels = model(images)
            loss_train = loss_function(predicted_labels, labels)
            loss_train.backward()
            optimizer.step()

            # Accumulate the loss and calculate the accuracy of predictions
            training_loss += loss_train.item()

            training_loop.set_postfix(train_loss="{:.8f}".format(
                training_loss / (train_iteration + 1)), val_loss="{:.8f}".format(validation_loss))

            # Update the tensorboard logger.
            # tb_logger.add_scalar(f'{name}/train_loss', loss.item(), epoch * len(dataloader_train) + train_iteration)

        """    Validation step for one batch of data    """
        val_loop = create_tqdm_bar(
            dataloader_validation, desc=f'Validation Epoch [{epoch+1}/{epochs}]')
        validation_loss = 0
        with torch.no_grad():
            for val_iteration, batch in val_loop:
                model.eval()  # Set the model to evaluation mode
                with torch.no_grad():
                    images, labels = batch
                    labels -= 1  # Change the labels to start from 0
                    labels = labels.type(torch.LongTensor)

                    images = images.to(device)
                    labels = labels.to(device)

                    # Forward pass
                    output = model(images)

                    # Calculate the loss
                    loss_val = loss_function(output, labels)

                validation_loss += loss_val.item()

                val_loop.set_postfix(val_loss="{:.8f}".format(
                    validation_loss/(val_iteration+1)))

                # Update the tensorboard logger.
                # tb_logger.add_scalar(f'{name}/val_loss', validation_loss/(val_iteration+1), epoch*len(dataloader_val)+val_iteration)

            # This value is for the progress bar of the training loop.
            validation_loss /= len(dataloader_validation)

        scheduler.step()
        print(scheduler.get_last_lr())

In [9]:
train_net(model, loss_function, DEVICE, dataloader_train,
          dataloader_validation, optimizer, hyper_parameters, name="default")

Validation Epoch [1/10]: 100%|████████████████████████████████████████████████████████████████| 440/440 [00:04<00:00, 105.22it/s, val_loss=1.64553674]


[0.001]


Validation Epoch [2/10]: 100%|████████████████████████████████████████████████████████████████| 440/440 [00:04<00:00, 105.99it/s, val_loss=1.60897828]


[0.001]


Validation Epoch [3/10]: 100%|████████████████████████████████████████████████████████████████| 440/440 [00:04<00:00, 106.23it/s, val_loss=1.60911665]


...

[0.001]


Validation Epoch [9/10]: 100%|████████████████████████████████████████████████████████████████| 440/440 [00:04<00:00, 104.60it/s, val_loss=1.60821076]


[0.001]


Validation Epoch [10/10]: 100%|███████████████████████████████████████████████████████████████| 440/440 [00:04<00:00, 105.48it/s, val_loss=1.60724354]

[0.001]
